In [1]:
# import liabries
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
#  src url and initial soup
# get url list for data scraping
url = 'https://www.who.int/csr/sars/country/en/'
bs0 = bs(requests.get(url).content,'html')
bs1 = bs0.find_all('a')
lst = list(map(lambda x: x if '/csr/sars/country/' in str(x) else 0, bs1))
who_main = 'http://www.who.int'
url_lst = list(map(lambda x: who_main+x,[lst[i]['href'] for i in range(len(lst)) if lst[i]!=0] ))

# url list to use for scraping
url_use = [i for i in url_lst if len(bs(requests.get(i).content,'html').find_all('tr'))!=0]

In [3]:
# normalize and retrieve list of date for df date column 
data_date = list(map(lambda x:re.findall('\d+_\d+_\d+',x)[0],url_use[3:]))
date_lst = ['2003_03_17','2003_03_18','2003_03_19']

#df date col data
data_date = date_lst+data_date


In [4]:
# define function to scrap data using url

def get_data(url,date):
    try:
        soup_table = bs(requests.get(url).content,'html').find_all('tr')
        rows = []
        for i in range(len(soup_table)):
            if '/b' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('b')])
            elif '/p' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('p')])
            elif '/td' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('td')]) 
        
        df = pd.DataFrame(data = rows[1:], columns=rows[0])
        df['Date'] = date
        result = [rows, df['Date']]
        return result
                # df.to_csv('/c/Users/sfy/Documents/Ironhack/Personal_Projects/data/%s')
    except:
        print('ValueError! Check source.',url)
        # raise ValueError('Check Column List Length')
        


In [49]:
# define function to clean data
def clean_col(x):
    cln_lst = x
    if type(cln_lst) == list:
        if len(cln_lst) != 0:
            for i in range(len(cln_lst[0])):
                if 'transmission' in cln_lst[0][i]:
                    cln_lst[0][i] = 'Local_Chain(s)_of_Transmission'
                elif 'Country' in cln_lst[0][i]:
                    cln_lst[0][i] = 'Country'
                elif ('Cumulative number' in cln_lst[0][i]) & ('Date for which cumulative number of cases is current' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Cumulative_Number_Of_Case(s)'
                elif ('Number of deaths' in cln_lst[0][i]) & ('Final Status/ Number of deaths' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Number_Of_Deaths'
                elif ('Number recovered' in cln_lst[0][i]) & ('Final Status/ Number recovered' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Number_Recovered'

    return cln_lst

def cln_trans(x):
    if 'None' in str(x):
        x = 'N'
    elif 'Yes' in str(x):
        x = 'Y'
    else:
        x = 'To be determined'
    return x

# # define function to get df
def get_df(data,date):
    df = pd.DataFrame(data=data[1:],columns=data[0])
    df['Date'] = date
    return df



In [6]:
origin = list(map(get_data,url_use,data_date))

ValueError! Check source. http://www.who.int/csr/sars/country/table/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_10/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_11/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_12/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_14/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_15/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_16/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_17/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_18/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_19/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_21/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_22/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_23/en/
Valu

In [7]:
data_origin = [origin[i][0] for i in range(len(origin)) if type(origin[i])==list]
data_df = list(map(clean_col,data_origin))

In [8]:
date_df = [origin[i][1] for i in range(len(origin)) if type(origin[i])==list]
lista = list(map(get_df, data_df, date_df))

In [15]:
dfa = pd.concat(lista)
dfa[dfa.columns[:5]]#['Cumulative_Number_Of_Case(s)']
dfa['Country'] = list(map(lambda x: 'China' if 'China' in x else x, dfa.Country))


,Country,Cumulative_Number_Of_Case(s),Number_Of_Deaths,Local_Chain(s)_of_Transmission,Date
0,Germany,2,0,None*,2003_03_18
1,Canada,8,2,Yes,2003_03_18
2,China \n +,,,,2003_03_18
3,Singapore,23,0,Yes,2003_03_18
4,Hong Kong Special Administrative Region of China,123,1**,Yes,2003_03_18
5,"Taiwan, China",3,0,Yes,2003_03_18
6,Thailand,1,0,None*,2003_03_18
7,Viet Nam,57,1,Yes,2003_03_18
8,Slovenia,1,0,None*,2003_03_18
9,United Kingdom,1,0,None*,2003_03_18


In [17]:
dfb = dfa[dfa.columns[:5]]
dfb['Local_Chain(s)_of_Transmission'] = list(map(cln_trans,dfb['Local_Chain(s)_of_Transmission']))
dfb

In [52]:
dfb.groupby(['Date','Country']).agg('sum')

Italy                                                    To be determined  
           Kuwait                                                   To be determined  
           Malaysia                                                 To be determined  
           Mongolia                                                 To be determined  
           New Zealand                                              To be determined  
           Philippines                                              To be determined  
           Republic of Ireland                                      To be determined  
           Republic of Korea                                        To be determined  
           Romania                                                  To be determined  
           Russian Federation                                       To be determined  
           Singapore                                                To be determined  
           South Africa                                             To be determined  
           Spain                                                    To be determined  
           Sweden                                                   To be determined  
           Switzerland                                              To be determined  
           Thailand                                                 To be determined  
           Total                                                    To be determined  
           United Kingdom                                           To be determined  
           United States^7                                          To be determined  
           Viet Nam                                                 To be determined  
2003_07_03 Australia                                                To be determined  
           Brazil                                                   To be determined  
           Canada^4                                                 To be determined  
           China                   To be determinedTo be determinedTo be determin...  
           Colombia                                                 To be determined  
           Finland                                                  To be determined  
           France                                                   To be determined  
           Germany                                                  To be determined  
           India                                                    To be determined  
           Indonesia                                                To be determined  
           Italy                                                    To be determined  
           Kuwait                                                   To be determined  
           Malaysia                                                 To be determined  
           Mongolia                                                 To be determined  
           New Zealand                                              To be determined  
           Philippines                                              To be determined  
           Republic of Ireland                                      To be determined  
           Republic of Korea                                        To be determined  
           Romania                                                  To be determined  
           Russian Federation                                       To be determined  
           Singapore                                                To be determined  
           South Africa                                             To be determined  
           Spain                                                    To be determined  
           Sweden                                                   To be determined  
           Switzerland                                              To be determined  
           Thailand                                                 To be determined  
         